In [113]:
import numpy as np
from scipy.linalg import expm, eigh

# Step 1: Define the Quantum System (Hamiltonian H)
# -------------------------------------------------------------------------------
# a random symmetric (Hermitian) matrix.
n_size=24
np.random.seed(42)
A = np.random.randn(n_size, n_size)
H = (A + A.T) / 2  
print("Hamiltonian H:\n", H)

Hamiltonian H:
 [[ 0.49671415 -0.34132351  0.49565341  0.74360191  0.03098345  0.2784475
   0.9195478   0.26102331 -0.12769032 -0.11513259 -0.62796922 -0.35914895
   0.26147707 -0.54393195 -1.27520751 -0.02147051 -0.88598189  0.21727148
  -0.58871545 -0.75966703  0.66886559 -0.43820943  0.16362535 -0.12848306]
 [-0.34132351  0.11092259 -1.45701687  0.97017084 -0.16979171 -0.6005406
   0.09005813  0.54927101 -0.629618   -0.64726477  0.35390424 -1.23431342
  -0.20691796 -0.57308024 -0.82478595  0.86480007  0.44443018  0.34290356
   0.30094699 -0.66817301 -1.21703122 -0.6034848  -0.6824777   0.65577153]
 [ 0.49565341 -1.45701687  0.32408397 -1.50241369 -0.33590427  1.0072353
  -0.10297559  0.02088284 -0.3330183  -0.39728796  0.41812536  1.30397822
  -0.34364824  0.55990992 -0.34670176 -0.65248339  0.5771409   1.03392745
   0.71517347  0.2249418  -0.62240515 -0.61875684  0.49999403  0.93781961]
 [ 0.74360191  0.97017084 -1.50241369  0.8219025  -0.07377003 -0.85042921
  -0.61434792 -1.40168

In [114]:
# Step 2: Choose an Initial Reference State
# -------------------------------------------------------------------------------
# Physically, this should overlap with Hartree-Fock.
phi0 = np.zeros(n_size)
phi0[0] = 1.0  # Start fully in the first basis state.
print("Initial state phi0:\n", phi0)

Initial state phi0:
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [191]:
############################################################
# Step 3: Evolve the State in Real Time
# -------------------------------------------------------------------------------
# Generate M time-evolved states: |n> = exp(-i H n dt) |phi0>
T = 10.0                    # Total time
M = 25                      # Time-steps in the simulation
delta_t = T/M               # Time-step size
states = []
for n in range(M):
    # Matrix exponential for time evolution
    U = expm(-1j * H * n * delta_t)
    states.append(U @ phi0)
states = np.array(states)  # shape (M, n_size): holds all |n> states

print(f"Generated {M} time-evolved states.")
print(np.shape(states))

Generated 25 time-evolved states.
(25, 24)


In [192]:
# ===============================================================================
# Step 4: Construct Filtered Fourier-Transformed States
# -------------------------------------------------------------------------------
# Choose J filter energies to target the spectrum region of interest.
J = 24
E_min, E_max = -7, 7
E_filters = np.linspace(E_min, E_max, J)
# Build filtered states: |ψ_j> = sum_n exp(i n E_j dt) |n>
psi = []
for E_j in E_filters:
    coeffs = np.exp(1j * np.arange(M) * E_j * delta_t)
    psi_j = np.sum(coeffs[:, None] * states, axis=0)
    psi.append(psi_j)
psi = np.array(psi)  # shape (J, n_size)
print(f"Constructed {J} filtered states.")
print(psi)

Constructed 24 filtered states.
[[ 7.26349240e-01-2.78730090e-01j -1.35560138e-01+2.09590765e-01j
  -1.67548797e-01+3.87645655e-01j -4.09859143e-02-1.02473925e-02j
  -1.25490773e-01+1.22202237e-01j -7.35797906e-03+6.48365093e-02j
  -5.49090522e-02+2.93811910e-01j  5.68281178e-02+5.10010303e-02j
   1.07420572e-04+2.80444228e-01j -8.50989919e-02+7.16154703e-02j
   2.44405296e-01-3.97020579e-01j  2.51922709e-01-1.64842535e-01j
  -2.26020543e-01+4.69353981e-01j -4.69406464e-02-1.83859159e-02j
   1.19665116e-01-5.00808149e-01j -1.12775778e-01+9.10394733e-02j
   9.41735332e-02-3.27476405e-01j  6.95187949e-02-2.34790183e-01j
   1.69781958e-01-4.59357601e-01j -7.76892965e-02+1.65212944e-01j
  -8.93413563e-02+9.36799996e-02j  9.85476517e-03-1.38957620e-01j
  -1.20493926e-03-2.01573046e-01j  1.97546671e-01-5.14094873e-01j]
 [ 6.41002357e-01-3.21893503e-01j  2.64356029e-02-1.76295156e-01j
  -4.33484888e-03-8.15971605e-02j -6.47801958e-02-3.33974430e-03j
  -6.64834011e-02+7.06714099e-02j -1.234548

In [193]:
# ===============================================================================
# Step 5: Compute Overlap and Hamiltonian Matrices
# -------------------------------------------------------------------------------
# S[j,j']: Overlap matrix in the filter basis
# F[j,j']: Hamiltonian in the filter basis
S = np.zeros((J, J), dtype=complex)
F = np.zeros((J, J), dtype=complex)
for j in range(J):
    for jp in range(J):
        S[j, jp] = np.vdot(psi[j], psi[jp])         # Eq: S_{jj'} = <ψ_j | ψ_{j'}>
        F[j, jp] = np.vdot(psi[j], H @ psi[jp])     # Eq: F_{jj'} = <ψ_j | H | ψ_{j'}>

print("Overlap matrix S:\n", S)
print("Hamiltonian matrix F:\n", F)

Overlap matrix S:
 [[ 2.76623788e+00+0.00000000e+00j  5.60229034e-02-1.25191931e-02j
   8.10794340e-01-3.81416058e-01j  1.14991802e+00-8.91647626e-01j
   7.89895934e-01-9.54369555e-01j  1.54177843e+00-3.02374385e+00j
  -9.33574544e-01+3.63079022e+00j -6.44820518e-02+2.02567294e+00j
   6.00864892e-02+3.15779869e-01j  3.34380141e-01+7.73814175e-01j
   4.95311811e-02+6.82569862e-02j  1.59611822e+00+1.40897716e+00j
   2.96940950e-01+1.63513731e-01j  3.92485111e+00+1.14348241e+00j
   2.15968199e+00+1.37635232e-01j  5.79047715e-01-9.11961433e-02j
  -8.25606575e-01+3.25995398e-01j -8.59756870e-01+5.83052549e-01j
   2.40764543e+00-2.55852741e+00j -1.12298303e-01+1.89409673e-01j
  -1.17572746e-01+3.60429464e-01j -3.77906678e-02+3.94655609e-01j
   6.89835872e-03+5.51718372e-02j  2.97807386e-01+8.30664758e-01j]
 [ 5.60229034e-02+1.25191931e-02j  2.04178578e+00+3.03576608e-17j
   1.73427731e+00-3.87551363e-01j  1.97851834e+00-9.30739931e-01j
   1.33830202e+00-1.03772077e+00j  2.70084930e+00-3.2632

In [194]:
# ===============================================================================
# Step 6: Solve the Generalized Eigenvalue Problem
# -------------------------------------------------------------------------------
# Finds E and c solving F c = E S c (standard in filter diagonalization)
# Eigenvalues E approximate eigenenergies of H near our chosen filter energies.
Eigvals, Eigvecs = eigh(F, S)
print("Computed Eigenvalues:\n", Eigvals)


Computed Eigenvalues:
 [-6.75956315 -4.80137205 -4.41267902 -4.16970499 -3.58792366 -3.17561386
 -2.87360609 -2.20427669 -1.7471599  -1.37078245 -1.07644908 -0.6869828
 -0.35845539  0.45915717  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]


In [195]:
# ===============================================================================
# Step 7 (Optional): Tune Parameters
# -------------------------------------------------------------------------------
# For larger or ill-conditioned problems, tune M, J, [E_min,E_max], or regularize S.
# (Can add: S += np.eye(J) * 1e-6 if S is near-singular)


In [196]:
# # Regularize if S is not positive definite
# epsilon = 1e-6
# S_reg = S + np.eye(J) * epsilon

# # Solve generalized eigenproblem
# Eigvals, Eigvecs = eigh(F, S_reg)


In [197]:
# ===============================================================================
# Step 8: Output Results
# -------------------------------------------------------------------------------
print("Approximated eigenvalues near selected filters:", Eigvals.real)
# Eigenvalues should be checked against the full matrix diagonalization.
# accurate Eigvals = eigh(H)[0]
accurate_Eigvals = eigh(H)[0]
print("Accurate eigenvalues from full diagonalization:", accurate_Eigvals)

Approximated eigenvalues near selected filters: [-6.75956315 -4.80137205 -4.41267902 -4.16970499 -3.58792366 -3.17561386
 -2.87360609 -2.20427669 -1.7471599  -1.37078245 -1.07644908 -0.6869828
 -0.35845539  0.45915717  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]
Accurate eigenvalues from full diagonalization: [-6.75956315 -4.80137205 -4.41267902 -4.16970497 -3.58792366 -3.17561363
 -2.87360608 -2.20427646 -1.74715978 -1.37078227 -1.07644621 -0.6869819
 -0.35845538  0.45915719  0.94152617  1.58575779  1.81856705  2.22254545
  3.03980074  3.88454002  4.22134803  4.92630963  5.80436737  6.53460187]
